In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

# Define the data directory and the necessary transformations
data_dir = '/content/drive/MyDrive/dataset'  # Update with the path to your data directory

# Define the transforms for the training and validation sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder(f'{data_dir}/{x}', data_transforms[x]) for x in ['train', 'val']}

# Define the dataloaders
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Define the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
model = models.resnet18(pretrained=True) # You can choose a different model if you prefer

# Modify the classifier according to your requirements
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # Assuming 3 classes in your case

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


num_epochs = 20
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs-1}')
    print('-' * 20)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


Epoch 0/19
--------------------
train Loss: 1.1655 Acc: 0.3816
val Loss: 1.0130 Acc: 0.5000
Epoch 1/19
--------------------
train Loss: 0.9066 Acc: 0.6053
val Loss: 0.2828 Acc: 1.0000
Epoch 2/19
--------------------
train Loss: 0.7939 Acc: 0.6974
val Loss: 0.1140 Acc: 1.0000
Epoch 3/19
--------------------
train Loss: 0.4813 Acc: 0.8158
val Loss: 0.0292 Acc: 1.0000
Epoch 4/19
--------------------
train Loss: 0.4836 Acc: 0.7632
val Loss: 0.0887 Acc: 1.0000
Epoch 5/19
--------------------
train Loss: 0.3088 Acc: 0.9211
val Loss: 0.0342 Acc: 1.0000
Epoch 6/19
--------------------
train Loss: 0.3967 Acc: 0.8553
val Loss: 0.1247 Acc: 1.0000
Epoch 7/19
--------------------
train Loss: 0.3048 Acc: 0.8553
val Loss: 0.4362 Acc: 0.8333
Epoch 8/19
--------------------
train Loss: 0.4431 Acc: 0.8289
val Loss: 0.5159 Acc: 0.8333
Epoch 9/19
--------------------
train Loss: 0.5767 Acc: 0.7500
val Loss: 0.2456 Acc: 0.8333
Epoch 10/19
--------------------
train Loss: 0.3449 Acc: 0.8947
val Loss: 0.0433

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/pred_img_type.path')
